## Simulation

In [7]:
#-----Import some essential packages

from __future__ import division
import hoomd
import hoomd.md

#-----Some important variables

box = 12.5;
copies = 10;
p_1 = 0.9;
p_2 = 1.8;
t_max = 5.0;

#-------Define the unit cell

hoomd.context.initialize("--mode=cpu", memory_traceback = True);

uc = hoomd.lattice.unitcell(N = 1,
                            a1 = [box, 0, 0],
                            a2 = [0, box, 0],
                            a3 = [0, 0, box],
                            dimensions = 3,
                            position = [[0,0,0]],
                            type_name = ['M'],
                            mass = [1.0],
                            moment_inertia = [[0,
                                               1/12*1.0*8**2,
                                               1/12*1.0*8**2]],
                            orientation = [[1, 0, 0, 0]]);

#-----From the lattice we obtain a snapshot and initialize.

snap = uc.get_snapshot()
snap.replicate(copies,copies,copies)
system = hoomd.init.read_snapshot(snap)

#-----Add the particle type for the constituent particles.

system.particles.types.add('A');

#-----Define each rigid body type in the local coordinate system of the body.

rigid = hoomd.md.constrain.rigid();
rigid.set_param('M', 
                types = ['A']*8,
                positions = [(-4,0,0),(-3,0,0),(-2,0,0),(-1,0,0),
                           (1,0,0),(2,0,0),(3,0,0),(4,0,0)]);

#-----Instruct the rigid constraint to create the constituent particles

rigid.create_bodies();

#-----Define the potential energy

nl = hoomd.md.nlist.cell()

lj = hoomd.md.pair.lj(r_cut = 3.5, nlist=nl)
lj.set_params(mode = 'shift')

#-----Define interaction with species in the binary mixture

lj.pair_coeff.set('M','M', epsilon = 1.0, sigma = 1.0)
lj.pair_coeff.set('A','A', epsilon = 1.0, sigma = 1.0)
lj.pair_coeff.set('M','A', epsilon = 1.0, sigma = 1.0)


#-----Select an standard integrator

hoomd.md.integrate.mode_standard(dt = 0.005);

#------Define group

mesogens = hoomd.group.rigid_center();

notice(2): Group "all" created containing 1000 particles
notice(2): constrain.rigid(): Creating 1000 rigid bodies (adding 8000 particles)
notice(2): Group "rigid_center" created containing 1000 particles


In [8]:
#----Integrate using NPT

    #----Define the ramps 
pressure_1 = hoomd.variant.linear_interp(points = [(0,0), (4.0e5, p_1)])
pressure_2 = hoomd.variant.linear_interp(points = [(5e5,p_1), (9e5, p_2)])

npt = hoomd.md.integrate.npt(group = mesogens, kT = t_max, tau = 1.0, tauP = 1.0, P = pressure_1);
npt.randomize_velocities(seed = 42)

#-----Write output and Run the Simulation

log_file = "T_" + str(t_max) + "_P_" + str(p_1) + "_initializing.log"
gsd_file = "T_" + str(t_max) + "_P_" + str(p_1) + "_initializing.gsd" 
meso_gsd_file = "T_CM_" + str(t_max) + "_P_" + str(p_1) + "_initializing.gsd"

log = hoomd.analyze.log(filename = log_file,
                         quantities = ['num_particles',
                                     'ndof',
                                     'translational_ndof',
                                     'rotational_ndof',
                                     'potential_energy',
                                     'kinetic_energy',
                                     'translational_kinetic_energy',
                                     'rotational_kinetic_energy',
                                     'temperature',
                                     'pressure',
                                      'volume'],
                         period = 1e3,
                         overwrite = True);
gsd = hoomd.dump.gsd(gsd_file,
               period = 1e3,
               group = hoomd.group.all(),
               overwrite = True); 
meso_gsd = hoomd.dump.gsd(meso_gsd_file,
               period = 1e3,
               group = mesogens,
               overwrite = True); 
hoomd.run(5e5)

notice(2): -- Neighborlist exclusion statistics -- :
notice(2): Particles with 0 exclusions             : 9000
notice(2): Neighbors included by diameter          : no
notice(2): Neighbors excluded when in the same body: yes
** starting run **
Time 00:00:10 | Step 304 / 500000 | TPS 30.2282 | ETA 04:35:30
Time 00:00:20 | Step 959 / 500000 | TPS 65.476 | ETA 02:07:01
Time 00:00:30 | Step 2548 / 500000 | TPS 158.727 | ETA 00:52:14
Time 00:00:40 | Step 4000 / 500000 | TPS 145.098 | ETA 00:56:58
Time 00:00:50 | Step 5284 / 500000 | TPS 128.385 | ETA 01:04:13
Time 00:01:00 | Step 6525 / 500000 | TPS 123.942 | ETA 01:06:21
Time 00:01:10 | Step 7558 / 500000 | TPS 103.285 | ETA 01:19:27
Time 00:01:20 | Step 8533 / 500000 | TPS 97.4377 | ETA 01:24:03
Time 00:01:30 | Step 9473 / 500000 | TPS 93.853 | ETA 01:27:06
Time 00:01:40 | Step 10448 / 500000 | TPS 97.4815 | ETA 01:23:41
Time 00:01:50 | Step 11464 / 500000 | TPS 101.431 | ETA 01:20:16
Time 00:02:00 | Step 12417 / 500000 | TPS 95.2589 | ETA

**ERROR**: Particle with unique tag 5427 has NaN for its position.


RuntimeError: Error computing cell list

In [ ]:
#-----Update npt parameters
npt.set_params(P = pressure_2)

#-----Write output and Run the Simulation

log_file = "T_" + str(t_max) + "_P_" + str(p_2) + "_ramp.log"
gsd_file = "T_" + str(t_max) + "_P_" + str(p_2) + "_ramp.gsd" 
meso_gsd_file = "T_CM_" + str(t_max) + "_P_" + str(p_2) + "_ramp.gsd"

log = hoomd.analyze.log(filename = log_file,
                         quantities = ['num_particles',
                                     'ndof',
                                     'translational_ndof',
                                     'rotational_ndof',
                                     'potential_energy',
                                     'kinetic_energy',
                                     'translational_kinetic_energy',
                                     'rotational_kinetic_energy',
                                     'temperature',
                                     'pressure',
                                      'volume'],
                         period = 1e3,
                         overwrite = True);
gsd = hoomd.dump.gsd(gsd_file,
               period = 1e3,
               group = hoomd.group.all(),
               overwrite = True); 
meso_gsd = hoomd.dump.gsd(meso_gsd_file,
               period = 1e3,
               group = mesogens,
               overwrite = True); 
hoomd.run(5e5)